# 이론
### LeNet-5
1988년 Yann LeCun에 의해 제안된 신경망 구조.   
MNIST 데이터셋 해결에 활용되었으며, CNN의 시초라고 할 수 있다.   
- avg pooling 적용하였고, 활성 함수는 tanh 적용.   
### AlexNet
LeNet보다 더 크고 복잡하지만 기본 구조는 동일하다.   
과적합을 막기 위해 굉장히 중요한 두 규제(regulation)기법을 적용했다.   
- 전결합층에 **드롭아웃**을 적용했음.
  - 매 이터레이션마다 랜덤으로 일정 비율의 뉴런을 비활성화하고 학습시켰다.
  - 과적합은 데이터셋 크기에 비해 학습시킬 파라미터가 많을 때 잘 발생한다.
  - 드롭아웃에서는 **더 적은 개수의 뉴런으로도 답을 얻게 유도**하여 과적합을 예방했다.
- **데이터 증강**기법을 적용하였다. 이는 **각 데이터의 본질을 유지하는 특정 변환**들을 가해 새로운 형태의 데이터를 만들어 내는 기법이다.
  - 이미지를 확대하거나 회전시키는 등의 변환을 거치면 학습 데이터셋이 많아진 것과 유사한 효과를 낸다.
### VGGNet
단순하면서 효과적인 구조로 현재까지 컴퓨터 비전의 가장 기초적인 CNN 구조로 활용되고 있다.
- 이전까지 다양한 크기의 커널들이 활용되었으나 VGGNet에서 처음으로 크기 3의 커널로 여러 번 합성곱을 진행하면 월등한 성능을 보임을 증명했다.
- 16개 혹은 19개 레이어를 갖는 버전이 존재한다.
- 풀링 이후 채널을 2배로 늘리는 것도 VGGNet 이후부터 정착되었다.
### GoogLeNet
- 다양한 크기의 커널을 사용해 다음 레이어로 넘기는 inception layer라는 레이어 구조를 제안했다.
- inception layer가 시작되기 전 일반 형태의 합성곱-풀링을 반복하는 구간을 **stem layer**라고 칭하며, 첫 레이어에서는 큰 커널과 2 stride를 적용해 입력 이미지 크기를 대폭 줄여준다.
- 처음부터 인셉션 레이어의 사용은 비효율적인 것을 관찰했다. 구글넷 이후 **다양한 신경망 구조에서도 stem layer를 적용**하고 있다.
- Global Average Pooling, GAP 개념을 도입했다.
  - 마지막 풀링을 마치고 일반적으로 flatten을 진행하고 다음 전결합층과 연결되는데 필요한 가중치가 너무 많았다.
  - 이에 마지막 **피처맵들을 피처맵과 같은 크기의 커널로 avg pooling**함으로써 1차원 피처를 만드는 GAP가 제안되었고 널리 활용되었다.
### ResNet
CNN 성능 확보에는 충분히 깊은 층이 꼭 필요하지만, 일정 이상으로 쌓게 되면 오히려 성능이 떨어지는 것을 확인했다.
- 입력을 그대로 출력으로 내보내는 아이덴티티 연산을 수행하는 레이어들을 추가해 성능이 같고 더 깊은 신경망을 만들었다.
- 두 번째 합성곱 이후에는 예전의 피처맵 값을 가져와 더해 activation을 수행했다.
### EfficientNet
**신경망의 크기(총 파라미터 수)를 제한**했을 때 어떻게 하이퍼파라미터를 조율해야 최적의 성능을 낼 수 있는지 연구된 바가 없었다.   
따라서 해당 연구에서는 제한된 신경망 크기 내 최고 성능 달성을 목표로 연구를 진행했다.   

## 전이학습
**다른 데이터셋에 대해 먼저 학습된 신경망의 가중치**를 **새로운** 데이터셋에 대해 학습하기 위한 **신경망의 가중치 초기값**으로 사용하는 학습 방식을 전이 학습, transfer learning이라고 한다.   
- 예를 들어 강아지 품종을 구분해야하는 데이터셋이 있을 경우, 아무것도 없는 신경망부터 시작하기보단 강아지, 고양이, 말을 **이미 구분할 수 있는 신경망에서부터 시작**해서 학습을 진행하는 것이 훨씬 유리할 것이다.   
- 전이 학습 방법에는 여러가지가 있으나 여기서 다룰 방식은 복사할 수 있는 가중치는 모두 복사하고 마지막에 분류를 위한 가중치만 새로 만들어 학습시키고자 한다.
  - 가중치 초기화 이후 학습 과정은 기존 CNN과 다를 바 없다.
  - 새로운 데이터셋에는 **클래스 개수가 달라지므로 신경망 맨 마지막 층은 새로 만들어**주어야 한다.

# 구현

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

#데이터 증강 및 정규화
data_transforms = {
    'train' : transforms.Compose([
      transforms.Resize(256),
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'val' : transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

데이터 증강 및 정규화에 대한 선언을 먼저 진행한다.   
**데이터 증강은 학습 데이터셋**에만 적용함에 유의할 것.

In [4]:
import os
from torchvision import datasets

#ImageFolder 기능을 활용해 파이토치 데이터셋 및 데이터로더 생성
data_dir = 'ants_bees'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=4) for x in ['train','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes

#GPU 사용 여부 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def train_model(model, criterion, optimizer, num_epochs=10):
  since = time.time()
  
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch,num_epochs-1))
    print('-'*10)
    
    #각 에폭은 학습 단계와 검증 단계를 거칩니다.
    for phase in ['train','val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
        
      running_loss = 0.0
      running_corrects = 0
      
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        #순전파
        #학습 시에만 연산 기록을 추적합니다.
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)
          
          #학습 단계인 경우 역전파 + 최적화
          if phase == 'train':
            loss.backward()
            optimizer.step()
          
        #통계
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]
      
      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
      
      #모델을 깊은 복사(deep copy)함
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    #가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model
  

학습 및 매 epoch 마다 검증까지 같이 수행하는 형태.

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import copy

model_ft = models.resnet18(pretrained=False, num_classes=2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

#모델 학습 및 평가
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.7951 Acc: 0.5041
val Loss: 0.6734 Acc: 0.5882

Training complete in 1m 0s
Best val Acc: 0.588235


In [8]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/daejin/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.6MB/s]


Epoch 0/9
----------
train Loss: 0.5903 Acc: 0.6926
val Loss: 0.2250 Acc: 0.9150

Training complete in 0m 60s
Best val Acc: 0.915033


ResNet 가중치 초기화 후 학습.   
가중치를 불러올 때는 이미지넷 학습 버전이므로 클래스 개수가 1000개로 고정된다.   
반대로 클래스 개수로 1000개가 아닌 데이터셋에 대한 전이 학습 시에는 **모델 생성 이후 마지막 전결합층을 다시 선언**해줘야 한다.

In [9]:
model_ft = models.efficientnet_v2_s(pretrained=False, num_classes=2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/daejin/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 3.6354 Acc: 0.5369
val Loss: 49.2248 Acc: 0.4837

Training complete in 4m 59s
Best val Acc: 0.483660


In [11]:
model_ft = models.efficientnet_v2_s(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier = nn.Linear(num_ftrs,2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.5676 Acc: 0.7418
val Loss: 0.3748 Acc: 0.8824

Training complete in 5m 32s
Best val Acc: 0.882353


EfficientNetV2는 classifier라는 이름의 마지막 전결합층을 갖고 있다.   
공식 코드에는 드롭아웃도 classifier에 존재하지만 이것은 제외하고 새로운 전결합층을 생성한다.

성능 결과들을 살펴보면 EfficientNetV2 전이학습 > ResNet 전이학습 > ResNet 초기화 후 학습 > EfficientNetV2 초기화 후 학습 순으로 나뉘어졌다.   
이처럼 새로운 이미지 분류 문제에 대해 **다양한 모델을 적용해 볼 필요**가 있으며, 다만 전이학습이 좋을지 초기화 후 학습이 좋을지는 **특정 모델로도 확인 가능**하다.
- resnet18로 전이학습 vs 초기화 후 학습을 판단한 후 구체적인 신경망 구조를 선택.
- 많은 경우 전이학습이 더 뛰어나지만, 꼭 그렇지는 않으므로 확인이 필요하다.